In [9]:
# Initial imports
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [10]:
# Load .env enviroment variables
load_dotenv()

True

In [11]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

'Dofr2kXxTQOvK7Txh1jBF03CWMbGtPWCD3C1hm5E'